In [570]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from matchup_generator import generate_matchups

In [571]:
training_df = generate_matchups(10)
training_df

/Users/anirudh/UIUC/go-mama-27/personal-projects/poke-predictor/ml-model/db.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,p1_id,p1_name,p1_hp,p1_attack,p1_defense,p1_sp_attack,p1_sp_defense,p1_speed,p1_primary_type,p1_secondary_type,...,opp6_name,opp6_hp,opp6_attack,opp6_defense,opp6_sp_attack,opp6_sp_defense,opp6_speed,opp6_primary_type,opp6_secondary_type,winner
0,65,alakazam,55,50,45,135,95,120,psychic,None,...,bulbasaur,45,49,49,65,65,45,grass,poison,2
1,75,graveler,55,95,115,45,45,35,rock,ground,...,ekans,35,60,44,40,54,55,poison,None,2
2,148,dragonair,61,84,65,70,70,70,dragon,None,...,kingler,55,130,115,50,50,75,water,None,2
3,71,victreebel,80,105,65,100,70,70,grass,poison,...,zapdos,90,90,85,125,90,100,electric,flying,1
4,70,weepinbell,65,90,50,85,45,55,grass,poison,...,growlithe,55,70,45,70,50,60,fire,None,1
5,70,weepinbell,65,90,50,85,45,55,grass,poison,...,venonat,60,55,50,40,55,45,bug,poison,1
6,151,mew,100,100,100,100,100,100,psychic,None,...,hitmonchan,50,105,79,35,110,76,fighting,None,2
7,91,cloyster,50,95,180,85,45,70,water,ice,...,slowbro,95,75,110,100,80,30,water,psychic,2
8,91,cloyster,50,95,180,85,45,70,water,ice,...,poliwrath,90,95,95,70,90,70,water,fighting,2
9,24,arbok,60,95,69,65,79,80,poison,None,...,mr-mime,40,45,65,100,120,90,psychic,fairy,1


In [572]:
X = training_df.drop(columns=["winner"])
y = training_df["winner"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [573]:
numeric_cols_x = X_train.select_dtypes(include='number').columns

sc = StandardScaler()
X_train_numeric = sc.fit_transform(X_train[numeric_cols_x])
X_test_numeric = sc.transform(X_test[numeric_cols_x])

In [574]:
X_train = torch.from_numpy(X_train_numeric.astype(np.float32))
X_test = torch.from_numpy(X_test_numeric.astype(np.float32))

''' 
    loss function needs values 0 and 1. 
    will make p1 win equal to 0 and p2 win equal to 1 
    (were originally 1 and 2 respectively)
'''
y_train = torch.from_numpy((y_train.to_numpy() - 1).astype(np.float32))
y_test = torch.from_numpy((y_test.to_numpy() - 1).astype(np.float32))

In [575]:
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

y_train = y_train.float()
y_test = y_test.float()

In [576]:
class LogisticRegression(nn.Module):
    def __init__(self, n):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n, 1)
        
    def forward(self, x):
        return self.linear(x)

In [577]:
n = X_train.shape[1]
model = LogisticRegression(n)

In [578]:
learning_rate = 0.01
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [579]:
num_epochs = 100

for epoch in range(num_epochs):
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

epoch: 10, loss = 0.1045
epoch: 20, loss = 0.0280
epoch: 30, loss = 0.0133
epoch: 40, loss = 0.0089
epoch: 50, loss = 0.0069
epoch: 60, loss = 0.0058
epoch: 70, loss = 0.0051
epoch: 80, loss = 0.0045
epoch: 90, loss = 0.0041
epoch: 100, loss = 0.0037


In [580]:
with torch.no_grad():
    y_pred_logits = model(X_test).squeeze()
    y_pred_probs = torch.sigmoid(y_pred_logits)
    y_pred_cls = (y_pred_probs >= 0.5).float()
    acc = (y_pred_cls == y_test).float().mean()
    print(f'accuracy = {acc:.4f}')

accuracy = 0.5000
